<a href="https://colab.research.google.com/github/sriku2412/AI-in-business/blob/main/MBAN6500X_class08_TextClass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Working With Text Data

The goal of this tutorial is to explore some of the main scikit-learn tools on a single practical task: analyzing a collection of text documents (newsgroups posts) on twenty different topics.

In this section we will see how to:

   * load the file contents and the categories
   * extract feature vectors suitable for machine learning
   * train a linear model to perform categorization
   * use a grid search strategy to find a good configuration of both the feature extraction components and the classifier


## Required packages

Scikit-klearn is already installed on Google Colab.
```
pip install -U sklearn
```


## Loading the 20 newsgroups dataset


The dataset is called "Twenty Newsgroups". Here is the official description, quoted from the [website](http://people.csail.mit.edu/jrennie/20Newsgroups/):

The 20 Newsgroups data set is a collection of approximately 20,000 newsgroup documents, partitioned (nearly) evenly across 20 different newsgroups. To the best of our knowledge, it was originally collected by Ken Lang, probably for his paper "Newsweeder: Learning to filter netnews," though he does not explicitly mention this collection. The 20 newsgroups collection has become a popular data set for experiments in text applications of machine learning techniques, such as text classification and text clustering.

In the following we will use the built-in dataset loader for 20 newsgroups from scikit-learn. Alternatively, it is possible to download the dataset manually from the website and use the [**`sklearn.datasets.load_files`**](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_files.html#sklearn.datasets.load_files) function by pointing it to the 20news-bydate-train sub-folder of the uncompressed archive folder.

In order to get faster execution times for this first example we will work on a partial dataset with only 4 categories out of the 20 available in the dataset:



In [44]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

We can now load the list of files matching those categories as follows:

In [45]:
from sklearn.datasets import fetch_20newsgroups
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

The returned dataset is a `scikit-learn` "bunch": a simple holder object with fields that can be both accessed as python dict keys or object attributes for convenience, for instance the target_names holds the list of the requested category names:

In [46]:
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

**Question 1**: How can we view the other attributes of ```twenty_train```?

In [47]:
twenty_train.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

In [48]:
dir(twenty_train)

['DESCR', 'data', 'filenames', 'target', 'target_names']

The files themselves are loaded in memory in the `data` attribute. For reference the filenames are also available:


In [49]:
print(twenty_train.filenames)

['/root/scikit_learn_data/20news_home/20news-bydate-train/comp.graphics/38440'
 '/root/scikit_learn_data/20news_home/20news-bydate-train/comp.graphics/38479'
 '/root/scikit_learn_data/20news_home/20news-bydate-train/soc.religion.christian/20737'
 ...
 '/root/scikit_learn_data/20news_home/20news-bydate-train/sci.med/58112'
 '/root/scikit_learn_data/20news_home/20news-bydate-train/sci.med/58578'
 '/root/scikit_learn_data/20news_home/20news-bydate-train/sci.med/58895']


**Question 2**: How many documents are there in the corpus?

In [50]:
len(twenty_train.filenames)

2257

Let’s print the first lines of the **first** loaded file (message).

**Question 3**: What is the subject of the message?

In [72]:
print(twenty_train.data[0])

From: sd345@city.ac.uk (Michael Collier)
Subject: Converting images to HP LaserJet III?
Nntp-Posting-Host: hampton
Organization: The City University
Lines: 14

Does anyone know of a good way (standard PC application/PD utility) to
convert tif/img/tga files into LaserJet III format.  We would also like to
do the same, converting to HPGL (HP plotter) files.

Please email any response.

Is this the correct group?

Thanks in advance.  Michael.
-- 
Michael Collier (Programmer)                 The Computer Unit,
Email: M.P.Collier@uk.ac.city                The City University,
Tel: 071 477-8000 x3769                      London,
Fax: 071 477-8565                            EC1V 0HB.



Supervised learning algorithms will require a category label for each document in the training set. In this case the category is the name of the newsgroup which also happens to be the name of the folder holding the individual documents.

For speed and space efficiency reasons `scikit-learn` loads the target attribute as an array of integers that corresponds to the index of the category name in the `target_names` list. The category integer id of each sample is stored in the `target` attribute:

**Question 4**: What is the label of the first message?

In [75]:
twenty_train.target[0]

1

**Question 5**: What is the value of `target_names` corresponding to the first message?

In [76]:
twenty_train.target_names[twenty_train.target[0]]

'comp.graphics'

To inspect the first 10 labels (tragets) try printing
```Python
twenty_train.target[:10]
```

In [77]:
twenty_train.target[:10]

array([1, 1, 3, 3, 3, 3, 3, 2, 2, 2])

It is possible to get back the category names as follows:
```Python
for t in twenty_train.target[:10]:
    print(twenty_train.target_names[t])
```

In [78]:
for t in twenty_train.target[:10]:
    print(twenty_train.target_names[t])

comp.graphics
comp.graphics
soc.religion.christian
soc.religion.christian
soc.religion.christian
soc.religion.christian
soc.religion.christian
sci.med
sci.med
sci.med


You might have noticed that the samples were shuffled randomly when we called ```fetch_20newsgroups(..., shuffle=True, random_state=42)```: this is useful if you wish to select only a subset of samples to quickly train a model and get a first idea of the results before re-training on the complete dataset later.

## Extracting features from text files



In order to perform machine learning on text documents, we first need to turn the text content into numerical feature vectors.

### Bags of Words (BoW)

The most intuitive way to do so is to use a bags of words representation:

1. Assign a fixed integer id to each word occurring in any document of the training set (for instance by building a dictionary from words to integer indices).

2. For each document ```i```, count the number of occurrences of each word w and store it in ```X[i, j]``` as the value of feature ```j``` where ```j``` is the index of word ```w``` in the dictionary.


The BoW representation implies that ```n_features``` is the number of distinct words in the corpus: this number is typically larger than 100,000.

If ```n_samples == 10000```, storing ```X``` as a NumPy array of type float32 would require 10000 x 100000 x 4 bytes = **4GB in RAM** which is barely manageable on a standard laptop.

Fortunately, **most values in ```X``` will be zeros** since for a given document less than a few thousand distinct words will be used. For this reason we say that bags of words are typically **high-dimensional** sparse datasets. We can save a lot of memory by only storing the non-zero parts of the feature vectors in memory.

```scipy.sparse``` matrices are data structures that do exactly this, and ```scikit-learn``` has built-in support for these structures.

## Tokenizing text with ```scikit-learn```

Text preprocessing, tokenizing and filtering of stopwords are all included in [**CountVectorizer**](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer), which builds a dictionary of features and transforms documents to feature vectors:

In [79]:
# v = [0, 0, 0, 0, 5]
#sparse_vec = (index=4, value=5)

In [80]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer()

X_train_counts = count_vect.fit_transform(twenty_train.data)

**Question 6**: What is the shape of `X_train_counts`?

In [81]:
X_train_counts.shape

(2257, 35788)

**Question 7**: What is the size of the vocabulary?


**Question 8**: What is the count of the most frequent word?

In [89]:
X_train_counts[:10,:10].todense()

matrix([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [101]:
index_of_max = X_train_counts.sum(axis=0).argmax()
[word for word, index in count_vect.vocabulary_.items() if index == index_of_max]

['the']

In [102]:
for k, v in count_vect.vocabulary_.items():
  if v== X_train_counts.sum(axis=0).argmax():
    print(k)

the


[**CountVectorizer**](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer) supports counts of N-grams of words or consecutive characters. Once fitted, the vectorizer has built a dictionary of feature indices:

In [104]:
count_vect.vocabulary_['algorithm']

4690

The index value of a word in the vocabulary is linked to its frequency in the whole training corpus.

### From occurrences to frequencies

Occurrence count is a good start but there is an issue: longer documents will have higher average count values than shorter documents, even though they might talk about the same topics.

To avoid these potential discrepancies it suffices to divide the number of occurrences of each word in a document by the total number of words in the document: these new features are called ```tf``` for Term Frequencies.

Another refinement on top of ```tf``` is to downscale weights for words that occur in many documents in the corpus and are therefore less informative than those that occur only in a smaller portion of the corpus.

This downscaling is called *tf–idf* for “Term Frequency times Inverse Document Frequency”.

Both **tf** and **tf–idf** can be computed as follows using [**TfidfTransformer**](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html#sklearn.feature_extraction.text.TfidfTransformer):

In [106]:
from sklearn.feature_extraction.text import TfidfTransformer

tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)

X_train_tf = tf_transformer.transform(X_train_counts)
X_train_tf.shape

(2257, 35788)

In the above example-code, we firstly use the ```fit(..)``` method to fit our estimator to the data and secondly the ```transform(..)``` method to transform our count-matrix to a tf-idf representation. These two steps can be combined to achieve the same end result faster by skipping redundant processing. This is done through using the ```fit_transform(..)``` method as shown below, and as mentioned in the note in the previous section:

In [107]:
tfidf_transformer = TfidfTransformer(use_idf=True)

X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(2257, 35788)

## Training a classifier

Now that we have our features, we can train a classifier to try to predict the category of a post. Let’s start with a [naïve Bayes](https://scikit-learn.org/stable/modules/naive_bayes.html#naive-bayes) classifier, which provides a nice baseline for this task. ```scikit-learn``` includes several variants of this classifier; the one most suitable for word counts is the multinomial variant:

In [57]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB()

clf.fit(X_train_tfidf, twenty_train.target)

MultinomialNB()

To try to predict the outcome on a new document we need to extract the features using almost the same feature extracting chain as before. The difference is that we call ```transform``` instead of ```fit_transform``` on the transformers, since they have already been fit to the training set:

In [58]:
docs_new = ['God is love', 'OpenGL on the GPU is fast']

X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

predicted = clf.predict(X_new_tfidf)

for doc, category in zip(docs_new, predicted):
    print('%r => %s' % (doc, twenty_train.target_names[category]))

'God is love' => soc.religion.christian
'OpenGL on the GPU is fast' => comp.graphics


## Building a pipeline

In order to make the vectorizer => transformer => classifier easier to work with, ```scikit-learn``` provides a [**Pipeline**](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline) class that behaves like a compound classifier:

In [59]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer(use_idf=True)),
    ('clf', MultinomialNB()),
])

The names ```vect```, ```tfidf``` and ```clf``` (classifier) are arbitrary. We will use them to perform grid search for suitable hyperparameters below. We can now train the model with a single command:

In [60]:
text_clf.fit(twenty_train.data, twenty_train.target)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB())])

## Evaluation of the performance on the test set

Evaluating the predictive accuracy of the model is equally easy:

In [61]:
import numpy as np
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=12)
docs_test = twenty_test.data
predicted = text_clf.predict(docs_test)

**Question 9**: What what accuracy did you get with Naive Bayes classifier?

In [108]:
from sklearn.metrics import accuracy_score
accuracy_score(y_pred = predicted, y_true = twenty_test.target)

0.9101198402130493

In [109]:
(twenty_test.target == predicted).mean()

0.9101198402130493

In [111]:
(twenty_train.target == 0).mean() , (twenty_test.target == 0).mean(), (twenty_train.target == 1).mean() , (twenty_test.target == 1).mean()

(0.21267168808152415,
 0.21238348868175766,
 0.25875055383252105,
 0.2589880159786951)

In [112]:
(twenty_test.target == 0).mean()

0.21238348868175766

We achieved ??% accuracy. Let’s see if we can do better with a linear [support vector machine (SVM)](https://scikit-learn.org/stable/modules/svm.html#svm), which is widely regarded as one of the best text classification algorithms (although it’s also a bit slower than naïve Bayes). We can change the learner by simply plugging a different classifier object into our pipeline:

In [113]:
from sklearn.linear_model import SGDClassifier
text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier(loss='hinge', penalty='l2',
                          alpha=1e-3, random_state=42,
                          max_iter=5, tol=None)),
])

text_clf.fit(twenty_train.data, twenty_train.target)

predicted = text_clf.predict(docs_test)

**Question 10**: What what accuracy did you get with the SVM classifier?

In [114]:
accuracy_score(y_pred=predicted,y_true=twenty_test.target)

0.9101198402130493

We achieved ??% accuracy using the SVM. ```scikit-learn``` provides further utilities for more detailed performance analysis of the results:

In [116]:
from sklearn import metrics
print(metrics.classification_report(twenty_test.target, predicted, target_names=twenty_test.target_names))

                        precision    recall  f1-score   support

           alt.atheism       0.95      0.80      0.87       319
         comp.graphics       0.87      0.98      0.92       389
               sci.med       0.94      0.89      0.91       396
soc.religion.christian       0.90      0.95      0.93       398

              accuracy                           0.91      1502
             macro avg       0.91      0.91      0.91      1502
          weighted avg       0.91      0.91      0.91      1502



In [117]:
metrics.confusion_matrix(twenty_test.target, predicted)

array([[256,  11,  16,  36],
       [  4, 380,   3,   2],
       [  5,  35, 353,   3],
       [  5,  11,   4, 378]])

As expected the confusion matrix shows that posts from the newsgroups on atheism and Christianity are more often confused for one another than with computer graphics.

## Parameter tuning using grid search

We’ve already encountered some parameters such as ```use_idf``` in the ```TfidfTransformer```. Classifiers tend to have many parameters as well; e.g., ```MultinomialNB``` includes a smoothing parameter ```alpha``` and ```SGDClassifier``` has a penalty parameter ```alpha``` and configurable loss and penalty terms in the objective function (see the module documentation, or use the Python ```help``` function to get a description of these).

Instead of tweaking the parameters of the various components of the chain, it is possible to run an exhaustive search of the best parameters on a grid of possible values. We try out all classifiers on either words or bigrams, with or without idf, and with a penalty parameter of either 0.01 or 0.001 for the linear SVM:

In [119]:
from sklearn.model_selection import GridSearchCV
parameters = {
    'vect__ngram_range': [(1, 1), (1, 2)], # uni gram and bi gram
    'tfidf__use_idf': (True, False), # using IDF inverse
    'clf__alpha': (1e-2, 1e-3), # controlles regularization
}

Obviously, such an exhaustive search can be expensive. If we have multiple CPU cores at our disposal, we can tell the grid searcher to try these eight parameter combinations in parallel with the ```n_jobs``` parameter. If we give this parameter a value of ```-1```, grid search will detect how many cores are installed and use them all:

In [120]:
gs_clf = GridSearchCV(text_clf, parameters, cv=5, n_jobs=-1)

The grid search instance behaves like a normal ```scikit-learn``` model. Let’s perform the search on a smaller subset of the training data to speed up the computation:

In [121]:
gs_clf = gs_clf.fit(twenty_train.data, twenty_train.target)

The result of calling ```fit``` on a ```GridSearchCV``` object is a classifier that we can use to ```predict```:

In [122]:
twenty_train.target_names[gs_clf.predict(['God is love'])[0]]

'soc.religion.christian'

The object’s ```best_score_``` and ```best_params_``` attributes store the best mean score and the parameters setting corresponding to that score:

In [123]:
gs_clf.best_score_

for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))

clf__alpha: 0.001
tfidf__use_idf: True
vect__ngram_range: (1, 1)


In [124]:
gs_clf.best_score_

0.9654425759865

**Question 11**: What what accuracy did you get with gridserach and the SVM classifier?


A more detailed summary of the search is available at ```gs_clf.cv_results_```.

The ```cv_results_``` parameter can be easily imported into pandas as a ```DataFrame``` for further inspection.

# Optional exercise: Language identification

* Write a text classification pipeline using a custom preprocessor and `CharNGramAnalyzer` using data from Wikipedia articles as training set.
* Evaluate the performance on some held out test set.

Start by downloading the dataset with
```Python
%run fetch_languagedata.py
```

In [126]:
%run fetch_languagedata.py

Exception: File `'fetch_languagedata.py'` not found.

Loaded the dataset with the ```load_files``` helper function from sklearn.

In [ ]:
from  sklearn.datasets import load_files
dataset = load_files('paragraphs', shuffle=True)